In [1]:
!pip install pyspark    #instalowanie PySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=d2db14737752552385ca8e26abe5886b0b28784c127a08163e109aeafdb5b7e6
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
!pip install Faker  #instalowanie Faker

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.4 MB/s eta 0:00:00


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .getOrCreate()

In [22]:
from pyspark import SparkFiles
from datetime import datetime
from pyspark.sql.types import DateType
from pyspark.sql import functions as sf
from pyspark.sql.window import Window


In [5]:
from faker import Faker
import json
import os
import shutil
import random


# Zadanie

Na podstawie zbiorów danych o oszustwach finansowych (fraudTrain) pobranych z https://www.kaggle.com/datasets/kartik2112/fraud-detection:

* Wykonaj podstawowe statystyki opisowe (min, max, avg) dla przykładowej ramki danych dla wszystkich pól numerycznych
* Sprawdź czy kobiety czy mężczyźni wykonują więcej transakcji i która z płci ma większą średnią wartość transakcji
* Sprawdź średnie i liczbę transakcji w rozbiciu na kategorie
* Sprawdź w którym mieście mieszka najwięcej kobiet
* Sprawdź w którym stanie mieszka najmniej mężczyzn

In [7]:
#wczytanie danych
spark.sparkContext.addFile('/content/drive/MyDrive/Rozproszone przetwarzanie danych/fraudTrain.csv')

fraud = spark.read.csv("file:///"+SparkFiles.get("fraudTrain.csv"), header = True, inferSchema = True)
fraud.show()    #oglądam jakie mam w ogóle dane


+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|_c0|trans_date_trans_time|             cc_num|            merchant|     category|   amt|      first|     last|gender|              street|                city|state|  zip|    lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  0|  2019-01-01 00:00:18|   2703186189

In [8]:
# Wykonaj podstawowe statystyki opisowe (min, max, avg) dla przykładowej ramki danych dla wszystkich pól numerycznych

# kolumny numeryczne:
#'cc_num','amt', 'zip','lat','long', 'city_pop', 'trans_num','unix_time','merch_lat','merch_long','is_fraud'

#Statystyki dla wszystkich kolumn numerycznych:
fraud.select('cc_num','amt', 'zip','lat','long', 'city_pop', 'trans_num','unix_time','merch_lat','merch_long','is_fraud').summary().show()


+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+
|summary|              cc_num|               amt|               zip|               lat|              long|         city_pop|           trans_num|           unix_time|         merch_lat|        merch_long|            is_fraud|
+-------+--------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+--------------------+------------------+------------------+--------------------+
|  count|             1296675|           1296675|           1296675|           1296675|           1296675|          1296675|             1296675|             1296675|           1296675|           1296675|             1296675|
|   mean|4.171920420797164...| 70.35103545607072| 48800.67109722945|38.537621614892025|-90.22633

In [9]:
# Czy kobiety czy mężczyźni wykonują więcej transakcji i która z płci ma większą średnią wartość transakcji

fraud.groupBy('gender').agg(
    sf.count('gender').alias('liczba transakcji'),
    sf.mean('amt').alias('średnia wartość transakcji')
).show()

#Odpowiedź: Kobiety wykonują więcej transakcji, średnia wartość transakcji jest podobna (nieco wyższa u mężczyzn)


+------+-----------------+--------------------------+
|gender|liczba transakcji|średnia wartość transakcji|
+------+-----------------+--------------------------+
|     F|           709863|         70.20032433018802|
|     M|           586812|         70.53334981220459|
+------+-----------------+--------------------------+



In [10]:
# Sprawdź średnie i liczbę transakcji w rozbiciu na kategorie

fraud.groupBy('category').agg(
    sf.count('category').alias('liczba transakcji'),
    sf.mean('amt').alias('średnia transakcji')
).show()


+--------------+-----------------+------------------+
|      category|liczba transakcji|średnia transakcji|
+--------------+-----------------+------------------+
|        travel|            40507|111.50471967808039|
|      misc_net|            63287| 80.86509488520576|
|   grocery_pos|           123638|116.96098594283274|
|     kids_pets|           113035| 57.53687052682819|
|  shopping_net|            97543| 88.42407635606853|
|   grocery_net|            45452| 53.67008448473113|
|   food_dining|            91461| 51.08690523829847|
| gas_transport|           131659| 63.43457181051072|
| personal_care|            90758|47.967678111020604|
|health_fitness|            85879| 54.18214022054315|
| entertainment|            94014| 64.21042142659638|
|          home|           123115|58.270138569629985|
|      misc_pos|            79655|  62.8909986818154|
|  shopping_pos|           116672| 79.77915532432803|
+--------------+-----------------+------------------+



In [11]:
# Sprawdź w którym mieście mieszka najwięcej kobiet
fraud.where(sf.col('gender') == "F").groupBy('city').agg(sf.count('gender').alias('ilość kobiet')).orderBy(sf.desc('ilość kobiet')).show()

#Odpowiedź: najwięcej kobiet mieszka w Phoenix


+------------+------------+
|        city|ilość kobiet|
+------------+------------+
|     Phoenix|        5075|
|    Meridian|        5060|
|      Thomas|        4634|
|     Houston|        4146|
|     Brandon|        4044|
|Philadelphia|        3517|
|       Uledi|        3123|
|     Elberta|        3123|
|    Superior|        3119|
|     Bradley|        3117|
|   Hinesburg|        3112|
|    Hinckley|        3110|
|     De Witt|        3106|
|  Wilmington|        3101|
|   Cottekill|        3095|
|   Littleton|        3090|
| East Canaan|        3090|
|      Conway|        3088|
|  Centerview|        3087|
|       Tulsa|        3086|
+------------+------------+
only showing top 20 rows



In [12]:
# Sprawdź w którym stanie mieszka najmniej mężczyzn

fraud.where(sf.col('gender') == "M").groupBy('state').agg(sf.count('gender').alias('ilość mężczyzn')).orderBy('ilość mężczyzn').show()

# Odpowiedź: najmniej mężczyzn mieska w stanach CT i HI


+-----+--------------+
|state|ilość mężczyzn|
+-----+--------------+
|   CT|          1032|
|   HI|          1032|
|   AZ|          1061|
|   DC|          2050|
|   AK|          2120|
|   WA|          2458|
|   ID|          2541|
|   ND|          2605|
|   UT|          3010|
|   SD|          4644|
|   OR|          5154|
|   NV|          5599|
|   VT|          6126|
|   SC|          6703|
|   NH|          6720|
|   CO|          7206|
|   WV|          7696|
|   MA|          7744|
|   KS|          8164|
|   WY|          8171|
+-----+--------------+
only showing top 20 rows



# Zadanie

* Zakładając, że dane są z lat 2019-2020, sprawdź czy jacyś niepełnoletni (<21 lat) wykonywali transakcje. Ile ich było?
* Zakładając, że noc to 00:00 – 06:00, ranek 06:00 – 12:00, południe 12:00 – 16:00, popołudnie 16:00 – 21:00, a reszta to noc – sprawdź kiedy najczęściej popełniane były przestępstwa
* Jak dużo miesięcy upłynęło pomiędzy pierwszym a ostatnim przestępstwem?

In [19]:
# Zakładając, że dane są z lat 2019-2020, sprawdź czy jacyś niepełnoletni (<21 lat) wykonywali transakcje. Ile ich było?

# kolumna 'dob' to data urodzenia
# data transakcji: 'trans_date_trans_time

# Konwertowanie kolumny "dob" na typ daty
df2_fraud = fraud.withColumn("dob", sf.col("dob").cast(DateType()))
df2_fraud = fraud.withColumn("trans_date_trans_time", sf.col("trans_date_trans_time").cast(DateType()))

# Obliczanie wieku na podstawie daty urodzenia
df2_fraud = df2_fraud.withColumn("age", ((sf.unix_timestamp("trans_date_trans_time") - sf.unix_timestamp("dob")) / (365 * 24 * 60 * 60)).cast("integer"))

count21 = df2_fraud.where(sf.col('age')<21).count()
print("liczba osób poniżej 21 roku życia, która robiła transakcje:",count21)


liczba osób poniżej 21 roku życia, która robiła transakcje: 50459


In [20]:
# Zakładając, że noc to 00:00 – 06:00, ranek 06:00 – 12:00, południe 12:00 – 16:00, popołudnie 16:00 – 21:00, a reszta to noc – sprawdź kiedy najczęściej popełniane były przestępstwa

#wyłuskane daty i czasu oraz przypisanie pory dnia do określonego czasu:
fraud2 = (
fraud
    .withColumn('time', sf.date_format('trans_date_trans_time', 'HH:mm:ss'))
    .withColumn('date', sf.to_date('trans_date_trans_time'))
    .withColumn('time of date', sf.when(sf.col('time').between('00:00:00', '06:00:00'), 'noc')
                                .when(sf.col('time').between('06:00:01', '12:00:00'), 'rano')
                                .when(sf.col('time').between('12:00:01', '16:00:00'), 'południe')
                                .when(sf.col('time').between('16:00:01', '21:00:00'), 'popołudnie')
                                .otherwise('noc'))
)

# pogrupowanie po porach dnia i posortowanie:
fraud2.groupBy('time of date').count().orderBy('time of date').show()

# Odp. Najwięcej przestępstw było w nocy.


+------------+------+
|time of date| count|
+------------+------+
|         noc|454438|
|  popołudnie|327829|
|    południe|260853|
|        rano|253555|
+------------+------+



In [21]:
# Jak dużo miesięcy upłynęło pomiędzy pierwszym a ostatnim przestępstwem?

fraud_min_max_date = fraud.where(sf.col('is_fraud') == 1).agg(
    sf.min('trans_date_trans_time').alias('min date'),
    sf.max('trans_date_trans_time').alias('max date')
)
fraud_min_max_date.show()

(
fraud_min_max_date
    .withColumn('ile miesiecy', sf.round(sf.months_between('max date', 'min date'),2))  #https://sparkbyexamples.com/pyspark/pyspark-difference-between-two-dates-days-months-years/
).show()


+-------------------+-------------------+
|           min date|           max date|
+-------------------+-------------------+
|2019-01-02 01:06:37|2020-06-21 03:59:46|
+-------------------+-------------------+

+-------------------+-------------------+------------+
|           min date|           max date|ile miesiecy|
+-------------------+-------------------+------------+
|2019-01-02 01:06:37|2020-06-21 03:59:46|       17.62|
+-------------------+-------------------+------------+



# Zadanie
Korzystając z pakietu Faker: https://faker.readthedocs.io/en/master/
wygeneruj 10000 plików JSON zawierających:
* Imię
* Nazwisko
* Datę Urodzenia
* Nr telefonu
* Miasto

Odczytaj te pliki za pomocą Sparka i sprawdź:
* Średnią długość imienia
* Miesiąc w którym urodziło się najwięcej osób

In [ ]:
# Generowanie plików z fakera

fake = Faker()

target_directory = "/content/drive/MyDrive/Rozproszone przetwarzanie danych/pliki_json"
os.makedirs(target_directory, exist_ok=True)

miasta = []
for i in range(20):
    miasto = fake.city()
    miasta.append(miasto)
print(miasta)

for i in range(10000):
    imie = fake.first_name()
    nazwisko = fake.last_name()
    data_urodzenia = fake.date_of_birth()
    nr_telefonu = fake.msisdn() #https://faker.readthedocs.io/en/master/providers/faker.providers.phone_number.html?highlight=cell_phone
    miasto = random.choice(miasta)


    # Tworzenie słownika z danymi
    data = {
        'imie': imie,
        "nazwisko": nazwisko,
        "data_urodzenia": str(data_urodzenia),  # Konwersja daty na format tekstowy
        "nr_telefonu": str(nr_telefonu),  # Konwersja daty na format tekstowy
        "miasto": miasto
    }

    # Tworzenie nazwy pliku
    filename = f"data_{i + 1}.json"

    # Zapisywanie danych do pliku JSON
    with open(filename, "w") as json_file:
        json.dump(data, json_file)

    # Przenoszenie plików do odpowiedniego folderu na dysku
    target_path = os.path.join(target_directory, filename)
    try:
        shutil.move(filename, target_path)
    except Exception as e:
        print(f"Przenoszenie pliku {filename} nie powiodło się: {e}")


['East Tonyafurt', 'South Elizabethton', 'Amandaborough', 'Wendyburgh', 'South Christopherstad', 'Boydberg', 'Mcknightshire', 'Collinston', 'Lake Sarah', 'Tracyfort', 'East Jamesmouth', 'Krististad', 'Jonesshire', 'Woodardport', 'New Carol', 'South Jessica', 'Nelsonville', 'Josephhaven', 'Mauricemouth', 'East Rickeyborough']


In [26]:
#Odczytanie wygenerowanych z fakera plików za pomocą Sparka

files = spark.read.json('/content/drive/MyDrive/Rozproszone przetwarzanie danych/pliki_json/data*.json')
files.show(vertical=True)


-RECORD 0------------------------------
 data_urodzenia | 1911-04-13           
 imie           | Kristopher           
 miasto         | South Christopher... 
 nazwisko       | Williamson           
 nr_telefonu    | 8957492539752        
-RECORD 1------------------------------
 data_urodzenia | 2017-10-03           
 imie           | Alexandra            
 miasto         | South Christopher... 
 nazwisko       | Richardson           
 nr_telefonu    | 9310951034181        
-RECORD 2------------------------------
 data_urodzenia | 1988-02-18           
 imie           | Christopher          
 miasto         | South Christopher... 
 nazwisko       | Alvarado             
 nr_telefonu    | 4785595762318        
-RECORD 3------------------------------
 data_urodzenia | 2005-07-01           
 imie           | Clayton              
 miasto         | South Christopher... 
 nazwisko       | Christensen          
 nr_telefonu    | 7509452430387        
-RECORD 4------------------------------


In [27]:
#Średnia długość imienia

# obliczenie długości imienia
df_with_length_name = (
files
    .withColumn('dlugosc imienia', sf.length(sf.col('imie')))
)

# średnia długość imienia:
df_with_length_name.agg(
    sf.mean('dlugosc imienia').alias('srednia dlugosc imienia')
).show()


+-----------------------+
|srednia dlugosc imienia|
+-----------------------+
|                 6.0216|
+-----------------------+



In [28]:
# Miesiąc w którym urodziło się najwięcej osób

df_month = files.withColumn('miesiac',sf.month('data_urodzenia'))

# liczba osób urodzona w każdym z miesiecy
df_count = df_month.groupBy('miesiac').count()

print('miesiąc, w którym urodziło się najwięcej osób:', df_count.orderBy(sf.desc('count')).first()[0])


miesiąc, w którym urodziło się najwięcej osób: 10


# Zadanie: 

Weź dane z ćwiczenia o Fakerze.
* Zapisz je do plików csv partycjonowanych po mieście.
* Weź jedno z wybranych miast i zapisz dane o datach urodzenia do pythonowej listy. Posortuj tę listę i wypisz ją na ekran

In [ ]:
#Zapisz je do plików csv partycjonowanych po mieście.
miasto = spark.read.json("/content/drive/MyDrive/Rozproszone przetwarzanie danych/pliki_json")

miasto.write.mode('overwrite').partitionBy("miasto").csv("/content/drive/MyDrive/Rozproszone przetwarzanie danych/miasta_part", header = True)

In [29]:
# Weź jedno z wybranych miast i zapisz dane o datach urodzenia do pythonowej listy. Posortuj tę listę i wypisz ją na ekran.

cities = spark.read.csv("/content/drive/MyDrive/Rozproszone przetwarzanie danych/miasta_part/", header = True).where(sf.col("miasto") =="Boydberg")
cities_local = cities.select('data_urodzenia').collect()
dates = [row.data_urodzenia for row in cities_local]
dates.sort()

print(dates)


['1908-02-14', '1908-03-01', '1908-08-27', '1908-11-13', '1909-01-19', '1909-02-23', '1909-03-01', '1909-04-14', '1909-07-10', '1909-09-01', '1910-04-16', '1910-05-08', '1910-05-28', '1910-09-20', '1910-11-17', '1910-12-21', '1911-03-31', '1911-05-04', '1911-05-07', '1911-06-28', '1912-04-01', '1912-05-14', '1912-07-21', '1912-08-30', '1913-06-14', '1914-02-11', '1914-08-20', '1915-02-19', '1915-04-04', '1915-06-12', '1915-08-25', '1915-10-03', '1915-10-10', '1915-10-19', '1916-01-04', '1916-03-04', '1916-09-30', '1916-11-13', '1917-04-22', '1918-01-16', '1918-02-12', '1918-03-20', '1918-05-15', '1918-10-10', '1919-05-30', '1919-06-14', '1919-08-01', '1919-11-12', '1919-12-05', '1920-06-23', '1920-09-19', '1920-10-29', '1920-12-07', '1921-02-15', '1921-03-03', '1921-06-13', '1921-09-08', '1921-12-28', '1922-01-15', '1922-02-07', '1922-06-05', '1922-08-02', '1922-11-17', '1923-01-26', '1923-02-13', '1923-03-02', '1923-06-18', '1923-08-03', '1923-09-17', '1923-11-16', '1924-01-18', '1924

# Zadanie

Korzystając z danych o oszustwach płatniczych (fraudTrain) i funkcji okienkowych,
odpowiedz na pytania:
* Wyznacz średnią, min oraz max wysokości transakcji w każdym stanie w każdym roku
* Wypisz trzy największe transakcje będące oszustwem w każdej kategorii

In [23]:
# Wyznacz średnią, min oraz max wysokości transakcji w każdym stanie w każdym roku

df2_fraud = fraud.withColumn('year',sf.year(fraud.trans_date_trans_time))

windowSpec = Window.partitionBy(["state","year"]).orderBy("year")
frauds_windowed = (
    df2_fraud
      .withColumn("row_no", sf.row_number().over(windowSpec))
      .withColumn("minimum", sf.min('amt').over(windowSpec))
      .withColumn("maksimum", sf.max('amt').over(windowSpec))
      .withColumn("avg", sf.avg('amt').over(windowSpec))
).select("state", "year", "minimum","maksimum","avg").where(sf.col("row_no") == 1)
frauds_windowed.show()


+-----+----+-------+--------+------------------+
|state|year|minimum|maksimum|               avg|
+-----+----+-------+--------+------------------+
|   AK|2019|    1.0| 3178.51| 70.98928947368414|
|   AK|2020|   1.08| 1037.47|61.198883333333406|
|   AL|2019|    1.0|10525.07|  65.4799780994419|
|   AL|2020|    1.0| 3960.95| 65.37444331123592|
|   AR|2019|    1.0|13536.84| 75.90609176641325|
|   AR|2020|   1.01| 4902.51| 73.67259067357476|
|   AZ|2019|   1.01| 2393.29|  74.6622212101496|
|   AZ|2020|   1.02| 2343.71| 76.02799351701773|
|   CA|2019|    1.0|11422.83| 73.34715364802607|
|   CA|2020|    1.0| 4986.55| 73.61200976098148|
|   CO|2019|   1.01| 8217.23| 78.17115193342086|
|   CO|2020|   1.01| 3348.47| 79.45144027812253|
|   CT|2019|   1.01| 8266.86| 66.49991835994196|
|   CT|2020|   1.01| 2107.32|60.171885844748985|
|   DC|2019|   1.01| 1550.36| 75.14123041650747|
|   DC|2020|    1.0| 5066.96| 80.79403614457831|
|   DE|2019|  10.93| 1012.77| 514.4933333333333|
|   FL|2019|    1.0|

In [24]:
# Wypisz trzy największe transakcje będące oszustwem w każdej kategorii

only_fraud = df2_fraud.where(sf.col('is_fraud') == 1)

windowSpec = Window.partitionBy('category').orderBy('amt')

only_fraud.withColumn('row_number', sf.row_number().over(windowSpec)) \
    .where(sf.col('row_number').between(1,3)).select('category','amt','year','row_number') \
    .show()
    

+--------------+------+----+----------+
|      category|   amt|year|row_number|
+--------------+------+----+----------+
| entertainment|337.24|2019|         1|
| entertainment|356.81|2019|         2|
| entertainment|366.76|2020|         3|
|   food_dining| 92.79|2019|         1|
|   food_dining| 95.94|2019|         2|
|   food_dining| 96.57|2019|         3|
| gas_transport|  5.93|2019|         1|
| gas_transport|  6.02|2019|         2|
| gas_transport|  6.23|2019|         3|
|   grocery_net|  5.97|2019|         1|
|   grocery_net|  6.42|2019|         2|
|   grocery_net|  6.48|2020|         3|
|   grocery_pos|237.04|2019|         1|
|   grocery_pos|238.78|2019|         2|
|   grocery_pos|239.07|2019|         3|
|health_fitness| 14.13|2020|         1|
|health_fitness| 15.18|2019|         2|
|health_fitness| 15.38|2019|         3|
|          home|177.41|2019|         1|
|          home|182.62|2019|         2|
+--------------+------+----+----------+
only showing top 20 rows

